# How to add control rules

In [27]:
from swmm_api.input_file.sections import Control
from swmm_api import SwmmInput

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


See the help for possible inputs:

In [29]:
help(Control)

Help on class Control in module swmm_api.input_file.sections.others:

class Control(swmm_api.input_file.helpers.BaseSectionObject)
 |  Control(name, conditions, actions, priority=0)
 |  
 |  Rules that control pump and regulator operation.
 |  
 |  Section:
 |      [CONTROLS]
 |  
 |  Purpose:
 |      Determines how pumps and regulators will be adjusted
 |      based on simulation time or conditions at specific nodes and links.
 |  
 |  Formats:
 |      Each control rule is a series of statements of the form:
 |      ::
 |  
 |          RULE    ruleID
 |          IF      condition_1
 |          AND     condition_2
 |          OR      condition_3
 |          AND     condition_4
 |          Etc.
 |          THEN    action_1
 |          AND     action_2
 |          Etc.
 |          ELSE    action_3
 |          AND     action_4
 |          Etc.
 |          PRIORITY value
 |  
 |  Remarks:
 |      `RuleID`: an ID label assigned to the rule.
 |      `condition_n`: a condition clause.
 |     

Here are some example control rules from the SWMM user manual (commandline section).

In [30]:
example_rule = """
; Simple time-based pump control
RULE R1
IF SIMULATION TIME > 8
THEN PUMP 12 STATUS = ON
ELSE PUMP 12 STATUS = OFF

; Multi-condition orifice gate control
RULE R2A
IF NODE 23 DEPTH > 12
AND LINK 165 FLOW > 100
THEN ORIFICE R55 SETTING = 0.5

RULE R2B
IF NODE 23 DEPTH > 12
AND LINK 165 FLOW > 200
THEN ORIFICE R55 SETTING = 1.0

RULE R2C
IF NODE 23 DEPTH <= 12
OR LINK 165 FLOW <= 100
THEN ORIFICE R55 SETTING = 0

; PID controller that attempts to keep Node 23’s depth at 12:
RULE PID_1
IF NODE 23 DEPTH <> 12
THEN ORIFICE R55 SETTING = PID 0.5 0.1 0.0

; Pump station operation with a main (N1A) and lag (N1B) pump
RULE R3A
IF NODE N1 DEPTH > 5
THEN PUMP N1A STATUS = ON

RULE R3B
IF PUMP N1A TIMEOPEN > 2:30
THEN PUMP N1B STATUS = ON
ELSE PUMP N1B STATUS = OFF

RULE R3C
IF NODE N1 DEPTH <= 0.5
THEN PUMP N1A STATUS = OFF
AND PUMP N1B STATUS = OFF
"""

We can create a control section by multiline strings like defined here as `example_rule`.

In [31]:
control_section = Control.create_section(example_rule)

We can print the `.inp`-file representation of any objet in the inp-data.

In [32]:
print(control_section['R1'].to_inp_line())

RULE R1
IF SIMULATION TIME > 8
THEN PUMP 12 STATUS = ON
ELSE PUMP 12 STATUS = OFF
PRIORITY 0



If we print the object itself, we see the underlying structure of the object.

In [33]:
control_section['R1']

Control(name='R1', conditions=[_Condition(logic='IF', kind='SIMULATION', label=nan, attribute='TIME', relation='>', value='8')], actions=[_Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='ON'), _Action(logic='ELSE', kind='PUMP', label='12', action='STATUS', relation='=', value='OFF')], priority=0)

Now we recreate the object printed above.

Here are three ways to input the parameters. All these ways result in the same object.

First we use helper-classes like `Control.LOGIC` to get autocompletion features in IDEs and we pass every parameter as positional argument.

In [34]:
Control(name='R99',
        conditions=[Control._Condition(Control.LOGIC.IF, Control.OBJECTS.SIMULATION, Control.ATTRIBUTES.TIME, '>', 8)],
        actions=[Control._Action(Control.LOGIC.THEN, Control.OBJECTS.PUMP, '12', Control.ATTRIBUTES.STATUS, '=', 'ON'),
                 Control._Action(Control.LOGIC.THEN, Control.OBJECTS.PUMP, '12', Control.ATTRIBUTES.STATUS, '=',
                                 'OFF')],
        priority=0)

Control(name='R99', conditions=[_Condition(logic='IF', kind='SIMULATION', label=nan, attribute='TIME', relation='>', value='8')], actions=[_Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='ON'), _Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='OFF')], priority=0)

Next we use helper-classes (like `Control.LOGIC`, `Control.ATTRIBUTES` or `Control.OBJECTS`) to get autocompletion features in IDEs and we pass every parameter as named argument (or keyword argument).

In [35]:
Control(name='R99',
        conditions=[Control._Condition(logic=Control.LOGIC.IF, kind=Control.OBJECTS.SIMULATION,
                                      attribute=Control.ATTRIBUTES.TIME, relation='>', value=8)],
        actions=[Control._Action(logic=Control.LOGIC.THEN, kind=Control.OBJECTS.PUMP, label='12',
                                 action=Control.ATTRIBUTES.STATUS, relation='=', value='ON'),
                 Control._Action(logic=Control.LOGIC.THEN, kind=Control.OBJECTS.PUMP, label='12',
                                 action=Control.ATTRIBUTES.STATUS, relation='=', value='OFF')],
        priority=0)

Control(name='R99', conditions=[_Condition(logic='IF', kind='SIMULATION', label=nan, attribute='TIME', relation='>', value=8)], actions=[_Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='ON'), _Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='OFF')], priority=0)

Last we use simply strings and floats and we pass every parameter as positional argument. This can also be done with named arguments.

In [36]:
control_object = Control(name='R99',
                         conditions=[Control._Condition('IF', 'SIMULATION', 'TIME', '>', 8)],
                         actions=[Control._Action('THEN', 'PUMP', '12', 'STATUS', '=', 'ON'),
                                  Control._Action('THEN', 'PUMP', '12', 'STATUS', '=', 'OFF')],
                         priority=0)
control_object

Control(name='R99', conditions=[_Condition(logic='IF', kind='SIMULATION', label=nan, attribute='TIME', relation='>', value='8')], actions=[_Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='ON'), _Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='OFF')], priority=0)

You could also reuse Conditions and Actions.

To change parameters you can edit the parameters of the object.

For example here we change for the first (and only) condition of the rule the start time from 8 to 7.

In [37]:
control_object.conditions[0].value = 7

And finally adding a control object to the inp-data

In [38]:
inp = SwmmInput()
inp.add_obj(control_object)
inp

{   'CONTROLS': {   'R99': Control(name='R99', conditions=[_Condition(logic='IF', kind='SIMULATION', label=nan, attribute='TIME', relation='>', value=7)], actions=[_Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='ON'), _Action(logic='THEN', kind='PUMP', label='12', action='STATUS', relation='=', value='OFF')], priority=0)}}

In [39]:
print(inp.to_string())



;;____________________________________________________________________________________________________
[CONTROLS]
RULE R99
IF SIMULATION TIME > 7
THEN PUMP 12 STATUS = ON
THEN PUMP 12 STATUS = OFF
PRIORITY 0


